In [1]:
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
from tqdm import tqdm
from transformers import AutoTokenizer, DataCollatorWithPadding

In [3]:
pip install --upgrade transformers

In [4]:
pip install accelerate -U

In [ ]:
pip install transformers[torch]

  Using cached accelerate-0.26.1-py3-none-any.whl (270 kB)


In [5]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00


In [6]:
pip -q install datasets

In [7]:
from datasets import load_dataset

dataset = load_dataset("SetFit/imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 25000
    })
})

In [9]:
dataset['train'][25]

{'text': "My girlfriend once brought around The Zombie Chronicles for us to watch as a joke. Little did we realize the joke was on her for paying £1 for it. While watching this film I started to come up with things I would rather be doing than watching The Zombie Chronicles. These included:<br /><br />1) Drinking bleach 2) Rubbing sand in my eyes 3) Writing a letter to Brad Sykes and Garrett Clancy 4) Re-enacting the American civil war 5) Tax returns 6) GCSE Maths 7) Sex with an old lady.<br /><br />Garrett Clancy, aka Sgt. Ben Draper wrote this? The guy couldn't even dig a hole properly. The best ting he did was kick a door down (the best part of the film). This was the worst film I have ever seen, and I've seen White Noise: The Light. Never has a film had so many mistakes in it. My girlfriend left it here, so now I live with the shame of owning this piece of crap.<br /><br />News just in: Owen Wilson watched this film and tried to kill himself. Fact.<br /><br />DO NOT WATCH",
 'label

In [10]:
dataset['train'][0].keys()

dict_keys(['text', 'label', 'label_text'])

In [11]:
checkpoint = "bert-base-uncased"                   #distilbert-base-uncased
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [13]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
tokenized_datasets['train'][0].keys()

dict_keys(['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'])

In [16]:
to_remove = list(tokenized_datasets['train'][0].keys())

to_remove.remove('input_ids')
to_remove.remove('attention_mask')
to_remove.remove('label')

tokenized_datasets = tokenized_datasets.remove_columns(to_remove)

In [17]:
tokenized_datasets['train'].features

{'label': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [18]:
#tokenized_datasets['train'][0]

In [19]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_datasets["test"],shuffle=True, batch_size=16, collate_fn=data_collator
)

In [20]:
for batch in tqdm(train_dataloader):
    break
{k: v.shape for k, v in batch.items()}

  0%|          | 0/1563 [00:00<?, ?it/s]


{'input_ids': torch.Size([16, 512]),
 'attention_mask': torch.Size([16, 512]),
 'labels': torch.Size([16])}

In [21]:
from transformers import AutoModelForSequenceClassification

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2, ignore_mismatched_sizes = True, id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
train_data_subset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))
test_data_subset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

In [23]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [24]:
import numpy as np
import evaluate

metric_acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric_acc.compute(predictions=predictions, references=labels)

In [25]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="best_model",
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon = 1e-08,
    disable_tqdm = False,
    do_eval = True,
    evaluation_strategy = "epoch",
    learning_rate = 2e-05,
    lr_scheduler_type = "linear",
    num_train_epochs = 5,
    optim = "adamw_hf",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    save_strategy = "epoch",
    seed = 42,
    weight_decay = 0.01,
    warmup_steps = 0,
    report_to = "none",
    save_steps = 32,
    eval_steps = 32,
    logging_steps = 32,
    load_best_model_at_end=True,
)

In [26]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_data_subset,
    eval_dataset=test_data_subset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [27]:
stats = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.671500,0.553544,0.770000
2,0.424500,0.269869,0.890000
3,0.214300,0.223038,0.910000
4,0.095900,0.240353,0.920000
5,0.066100,0.296664,0.880000


In [28]:
from pprint import pprint
pprint(trainer.evaluate())

{'epoch': 5.0,
 'eval_accuracy': 0.91,
 'eval_loss': 0.22303780913352966,
 'eval_runtime': 3.1341,
 'eval_samples_per_second': 31.907,
 'eval_steps_per_second': 2.233}


In [30]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="/content/best_model/checkpoint-160", tokenizer=tokenizer)

In [34]:
text_1 = "This is awesome film. I did liked the film."

In [35]:
result = classifier(text_1)

In [36]:
print(result)

[{'label': 'POSITIVE', 'score': 0.9804131388664246}]
